In [ ]:
from statbank import StatbankClient
import pandas as pd
import dapla as dp
from io import StringIO

fileclient = dp.FileClient()
statclient = StatbankClient()

In [ ]:
df_stat = statclient.apidata("11721", {
  "query": [
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2022M10",
        ],
      },
    },
  ],
  "response": {
    "format": "json-stat2",
  },
}, include_id=True).drop(columns=["makrostørrelse", "statistikkvariabel"])
col_order = df_stat["ContentsCode"].unique().tolist()
mnr = df_stat.pivot_table(values="value", columns="ContentsCode", index=["Makrost", "måned"])
mnr = mnr[col_order].reset_index()
#mnr

In [ ]:
desc = statclient.get_description("11721")

In [ ]:
total_cols = (len(desc.variables[0]["variabler"]) +
len(desc.variables[0]["statistikkvariabler"]) +
len(desc.variables[0]["null_prikk_missing"]))
if len(mnr.columns) != total_cols:
    for colnum in range(total_cols - len(mnr.columns)):
        mnr[f"prikkecol_{colnum+1}"] = ""
#mnr

In [ ]:
data = desc.transferdata_template(mnr)

In [ ]:
data = desc.round_data(data)

In [ ]:
data["knrmakrohovmnd1.dat"]

In [ ]:
errors = statclient.validate(data, "11721")

In [ ]:
list(errors.keys())

In [ ]:
statclient.transfer(data, "11721")